# Report handler examples

Install leakpro as ``` pip install -e /path/to/leakpro ```

### Synthetic examples

In [1]:
import os
import sys

import pandas as pd

sys.path.append("../..")

from leakpro.synthetic_data_attacks.anomalies import return_anomalies
from leakpro.synthetic_data_attacks.inference_utils import inference_risk_evaluation
from leakpro.synthetic_data_attacks.linkability_utils import linkability_risk_evaluation
from leakpro.synthetic_data_attacks.singling_out_utils import singling_out_risk_evaluation

#Get ori and syn
n_samples = 100
DATA_PATH = "../synthetic_data/datasets/"
ori = pd.read_csv(os.path.join(DATA_PATH, "adults_ori.csv"), nrows=n_samples)
syn = pd.read_csv(os.path.join(DATA_PATH, "adults_syn.csv"), nrows=n_samples)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# syn_anom = return_anomalies(df=syn, n_estimators=1000, n_jobs=-1, verbose=True)
# print("Syn anom shape",syn_anom.shape)

In [3]:
# # Create a singling-out result
# sin_out_res = singling_out_risk_evaluation(
#     dataset = "adults",
#     ori = ori,
#     syn = syn_anom,
#     n_attacks = syn_anom.shape[0]
# )

In [4]:
# # Create linkability result
# link_res = linkability_risk_evaluation(
#     dataset = "adults",
#     ori = ori,
#     syn = syn_anom,
#     n_samples = syn_anom.shape[0],
#     n_attacks = 100
# )

In [5]:
# # # Create base-case inference result
# inf_res = inference_risk_evaluation(
#     dataset = "adults",
#     ori = ori,
#     syn = syn_anom,
#     worst_case_flag = False,
#     n_attacks = syn_anom.shape[0]
# )

# # # Create worst-case inference result
# inf_res_worst = inference_risk_evaluation(
#     dataset = "adults",
#     ori = ori,
#     syn = syn_anom,
#     worst_case_flag = True,
#     n_attacks = syn_anom.shape[0]
# )

### Gradient inversion example

In [6]:
sys.path.append("../gia/inverting_cifar10_1_image/")
from cifar import get_cifar10_loader
from model import ResNet
from torchvision.models.resnet import BasicBlock

from leakpro.attacks.gia_attacks.invertinggradients import InvertingConfig
from leakpro.fl_utils.gia_train import train
from leakpro.run import run_inverting

model = ResNet(BasicBlock, [5, 5, 5], num_classes=10, base_width=16 * 10)
client_dataloader, data_mean, data_std = get_cifar10_loader(num_images=1, batch_size=1, num_workers=2)

# Meta train function designed to work with GIA
train_fn = train

# Baseline config
configs = InvertingConfig()
configs.at_iterations = 1000 # Decreased from 8000 to avoid GPU memory crash

name = "my_gia_results"
GIA_result = run_inverting(model, client_dataloader, train_fn, data_mean, data_std, configs, experiment_name=name, save=False)


Files already downloaded and verified


2025-02-27 12:23:25,328 INFO     Inverting gradient initialized.


In [12]:
# # res = next(GIA_result)
# print(res)
print(type(GIA_result))


<class 'generator'>


In [21]:
GIA_result.__dir__()

['__repr__',
 '__getattribute__',
 '__iter__',
 '__next__',
 '__del__',
 'send',
 'throw',
 'close',
 'gi_frame',
 'gi_code',
 '__name__',
 '__qualname__',
 'gi_yieldfrom',
 'gi_running',
 '__doc__',
 '__new__',
 '__hash__',
 '__str__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

### Membership Inference Attack, CIFAR example

In [ ]:
# import os
# import sys
# import yaml

# project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
# sys.path.append(project_root)

In [ ]:
# from mia_utils.utils.cifar_data_preparation import get_cifar_dataloader
# from mia_utils.utils.cifar_model_preparation import ResNet18, create_trained_model_and_metadata


# # Load the config.yaml file
# with open('mia_utils/train_config.yaml', 'r') as file:
#     train_config = yaml.safe_load(file)

# # Generate the dataset and dataloaders
# path = os.path.join(os.getcwd(), train_config["data"]["data_dir"])

# train_loader, test_loader = get_cifar_dataloader(path, train_config)

In [ ]:
# # Train the model
# if not os.path.exists("target"):
#     os.makedirs("target")
# if train_config["data"]["dataset"] == "cifar10":
#     num_classes = 10
# elif train_config["data"]["dataset"] == "cifar100":
#     num_classes = 100
# else:
#     raise ValueError("Invalid dataset name")

# model = ResNet18(num_classes = num_classes)
# train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
#                                                                                train_loader, 
#                                                                                test_loader, 
#                                                                                train_config)

##### Run the MIA attack

In [ ]:
# from mia_utils.cifar_handler import CifarInputHandler

# from leakpro import LeakPro

# # Read the config file
# config_path = "mia_utils/audit.yaml"

# # Prepare leakpro object
# leakpro = LeakPro(CifarInputHandler, config_path)

# # Run the audit 
# mia_results = leakpro.run_audit(return_results=True)

In [ ]:
# Import and initialize ReportHandler
from leakpro.reporting.report_handler import ReportHandler

# Set report_dir to "./leakpro_output/results" to the results to a local results folder
#    or don't use the report_dir argument to let the ReportHandler find an already
#    existing results folder
report_handler = ReportHandler(report_dir="./leakpro_output/results")

# Save Synthetic results using the ReportHandler
report_handler.save_results(attack_name="singling_out", result_data=sin_out_res)
report_handler.save_results(attack_name="linkability_risk", result_data=link_res)
report_handler.save_results(attack_name="inference_risk_base", result_data=inf_res)
report_handler.save_results(attack_name="inference_risk_worst", result_data=inf_res_worst)

# Save GIA results using report handler
report_handler.save_results(attack_name="gia", result_data=GIA_result)

# Save MIA resuls using report handler
for res in mia_results:
    report_handler.save_results(attack_name=res.attack_name, result_data=res, config=res.configs)

2024-12-18 01:00:10,883 INFO     Initializing report handler...
2024-12-18 01:00:10,885 INFO     report_dir set to: ./leakpro_output/results


In [2]:
# Simply create a report by collecting and compiling latex text
report_handler.create_report()

2024-12-18 01:00:22,740 INFO     No results of type GIAResults found.
2024-12-18 01:00:22,743 INFO     No results of type SinglingOutResults found.
2024-12-18 01:00:22,744 INFO     No results of type InferenceResults found.
2024-12-18 01:00:22,745 INFO     No results of type LinkabilityResults found.
2024-12-18 01:00:31,133 INFO     PDF compiled


<Figure size 640x480 with 0 Axes>

In [ ]:
#### Have more granular control

# Use the ReportHandler and load all the saved results
report_handler.load_results()

# Create results and collect corresponding latex texts. Use different methods to create results for a specific type
# report_handler.create_results_mia()
# report_handler.create_results_gia()
# report_handler.create_results_syn()
report_handler.create_results_all()

# Create the report by compiling the latex text
report_handler.create_report()

2024-12-18 01:00:42,989 INFO     No results of type GIAResults found.
2024-12-18 01:00:42,992 INFO     No results of type SinglingOutResults found.
2024-12-18 01:00:42,993 INFO     No results of type InferenceResults found.
2024-12-18 01:00:42,994 INFO     No results of type LinkabilityResults found.
2024-12-18 01:00:51,396 INFO     PDF compiled


<Figure size 640x480 with 0 Axes>